In [55]:
import sys
import json
import tifffile
import collections

In [3]:
sys.path.insert(0, './opencell/')
import opencell

In [7]:
from opencell.imaging.images import RawPipelineTIFF

In [41]:
filepath = '/Users/keith.cheveralls/image-data/MMStack_1691-H8-35.ome.tif'
filepath = '/Users/keith.cheveralls/image-data/H6_10_PCNA.ome.tif'

In [42]:
tiff = RawPipelineTIFF(filepath)
tiff.parse_micromanager_metadata()
tiff.validate_micromanager_metadata()
tiff.split_channels()

In [43]:
tiff = tifffile.TiffFile(filepath)
len(tiff.pages)

102

In [44]:
# MicroManager tag attributes
mm_tag = {
    'code': 'MicroManagerMetadata',
    'dtype': 's',
    'count': 0,
    'value': None,
    'writeonce': False
}

# the order matters (because the tag is passed to TiffWriter.save as a tuple)
mm_tag = collections.OrderedDict(**mm_tag)

downsample_by = 10
pages_out = []
tags_out = []
for ind, page in enumerate(tiff.pages):
    # use the mm tag from the first page for every page
    mm_tag['value'] = json.dumps(tiff.pages[0].tags['MicroManagerMetadata'].value)
    tags_out.append(tuple(mm_tag.values()))
    pages_out.append(page.asarray()[::downsample_by, ::downsample_by])

In [45]:
tiff_out = tifffile.TiffWriter('/Users/keith.cheveralls/image-data/test_raw_stack.tif')
for page, tag in zip(pages_out, tags_out):
    tiff_out.save(page, extratags=[tag], contiguous=False)
tiff_out.close()

In [46]:
tiff = tifffile.TiffFile('/Users/keith.cheveralls/image-data/test_raw_stack.tif')
len(tiff.pages)

102

In [47]:
tiff.pages[1].tags['MicroManagerMetadata'].value['SliceIndex']

0

In [48]:
tiff = RawPipelineTIFF('/Users/keith.cheveralls/image-data/test_raw_stack.tif')
tiff.parse_micromanager_metadata()
tiff.validate_micromanager_metadata()
tiff.split_channels()
tiff.did_split_channels

EVENT: There was no IJMetadata tag found on the first page
EVENT: Unexpected slice_ind increment 0 for channel_ind 0


True

In [49]:
tiff.project_stack('405', axis='z')

In [ ]:
tiff.calculate_z_profiles('405')

In [53]:
stacks, result = tiff.align_cell_layer(-5, 5, .5)

In [54]:
result

{'offset_488_405': 2, 'crop_window': [3, 23], 'cell_layer_center': 13.0}

In [40]:
tiff.events

[{'message': 'There was no IJMetadata tag found on the first page',
  'timestamp': '2020-02-03 11:23:32'},
 {'message': 'Unexpected slice_ind increment 0 for channel_ind 0',
  'timestamp': '2020-02-03 11:23:32'}]